# TEST OF METRIC TOOLS

In [ ]:
import os
import radon.visitors
import radon.metrics
from radon.visitors import ComplexityVisitor

path = os. getcwd()

pythonFiles = [os.path.join(root, name)
             for root, dirs, files in os.walk(path)
             for name in files
             if (name.endswith((".py")) and # only python files
                 name !="__init__.py" and # remove the module initialisation 
                 not("test" in name.lower()) # remove test file
                 )]

print(pythonFiles)
#test namedtuple
from collections import namedtuple

ComplexityMetrics = namedtuple("CodeMetrics",["path","name","complexity","complexityLetter","is_method","class_name"])

LineMetric = namedtuple("LineMetric",["path","nbLines","nbLineSource","nbBlankLine","nbCommentLines","nbMultiCommentLines","maintenance_index","maintenance_index_letter"])

metric_complexity = []
metric_LOC        = []

# analyse file
for pythonFile in pythonFiles:
    with open(pythonFile, 'r') as file:
        data = file.read()

    res = ComplexityVisitor.from_code(data)

    #function
    list_function = res.functions

    for item in list_function :
        metric_complexity.append(ComplexityMetrics(
            path = pythonFile,
            name = item.name,
            complexity = item.complexity,
            complexityLetter= radon.complexity.cc_rank(item.complexity),
            is_method = False,
            class_name = item.classname))
    
    # Classes
    list_classes = res.classes
    for classItem in list_classes:
        #print(classItem)
        for methodItem in classItem.methods:
            metric_complexity.append(ComplexityMetrics(
            path = pythonFile,
            name = methodItem.name,
            complexity = methodItem.complexity,
            complexityLetter= radon.complexity.cc_rank(methodItem.complexity),
            is_method = True,
            class_name = methodItem.classname))

    import radon.raw

    loc_metric = radon.raw.analyze(data)
    MI_metric = radon.metrics.mi_visit(data,multi=True)


    metric_LOC.append(LineMetric(path=pythonFile,nbLines=loc_metric.loc,nbLineSource=loc_metric.sloc,nbBlankLine=loc_metric.blank,nbCommentLines=loc_metric.comments,nbMultiCommentLines=loc_metric.multi,maintenance_index=MI_metric,maintenance_index_letter=radon.metrics.mi_rank(MI_metric)))
    
print(metric_LOC)
